# **Monte Carlo Modeling Notebook**

##### This notebook contains our interactive Monte Carlo model. We will fit a Student's t-distribution to our trading data in order to forecast 1-year price predictions.

The Student's t-distribution reflects the fatter tails commonly seen in real-world trading data (Mandelbrot & Hudson, The Misbehavior of Markets, 2004). For this reason, we have chosen this direction rather than a simpler Normal distribution.

First, we will import necessary packages.

Additionally, you will need to retrieve a free API key from alphavantage, and enter it in the `apiKey` variable name. This key can be retrieved from:

https://www.alphavantage.co/

In [ ]:
import requests, pandas as pd, numpy as np
from scipy import stats
from datetime import datetime, timedelta
import plotly.graph_objects as go

In [30]:
apiKey = "YOUR_KEY_HERE"

For the simulation historical data, we're collecting three years of daily closing prices, for the stock ticker entered. Historcal prices are fetched from the Alpha Vantage API.

In [21]:
sym = input("Enter a stock ticker: ").strip().upper()
pars = {"function": "TIME_SERIES_DAILY", "symbol": sym, "outputsize": "full", "apikey": apiKey}
r = requests.get("https://www.alphavantage.co/query", params = pars)
d = r.json()
if "Time Series (Daily)" not in d:
    print("Error or no data returned")
else:
    f = pd.DataFrame.from_dict(d["Time Series (Daily)"], orient = "index").apply(pd.to_numeric)
    f.index = pd.to_datetime(f.index); f = f.sort_index()
    f.columns = ["Open","High","Low","Close","Volume"]
    f = f[f.index >= datetime.now() - timedelta(days = 3 * 365)]
    f["Ret"] = f["Close"].pct_change()
    ret = f["Ret"].dropna()

The dataset is trimmed to the prior three years of data. From the trimmed dataset, we pull only the daily percentage returns. These percentages will be foundational for fitting our Students t-distribution.

In [29]:
df, loc, sc = stats.t.fit(ret)
spot = f["Close"].iloc[-1]
days, showRuns, bigRuns = 252, 25, 100000

paths = np.array([[spot * np.prod(1 + stats.t.rvs(df, loc, sc, size = item)) for item in range(1, days + 1)] for _ in range(showRuns)]).T

finals = np.array([spot * np.prod(1 + stats.t.rvs(df, loc, sc, size = days)) for _ in range(bigRuns)])
low, high = np.percentile(finals, [1.25, 98.75])
trim = finals[(finals >= low) & (finals <= high)]

Simulating daily price paths is the first goal of our analysis. 25 total, daily paths will be used for the line chart pathing. Then, we will build a larger simulation (100,000 total samples) for a more concrete histogram plot.

Each line chart path will move sequentially over 252 total trading days, which lines up with roughly one calendar year.

Our next plot has 25 different simulated "paths". These trajectories help show what a real price track for that stock may look like.

Each line represents a single potential trajectory of your picked stock price. You can hover over the line to show the sample number, forecasted price, and day of that datapoint.

In [26]:
lineFig = go.Figure()
for iter in range(simsCount):
    lineFig.add_trace(go.Scatter(
        x = np.arange(1, daysCount + 1),
        y = pathsArray[:, iter],
        customdata = np.full(daysCount, iter + 1),
        mode = "lines",
        line = dict(width = 1, color = "rgba(0,0,0,0.25)"),
        hovertemplate = "<b>Sample Number:</b> %{customdata}<br><b>Day:</b> %{x}<br><b>Price:</b> $%{y:.2f}<extra></extra>"
    ))
lineFig.update_layout(
    title = symbolText + " Monte Carlo Simulated Price Paths (1 Year, 25 Runs)",
    xaxis_title = "Days",
    yaxis_title = "Price ($)",
    template = "plotly_white",
    showlegend = False,
    font = dict(family = "Arial", size = 14)
)
lineFig.show()



Lastly, we will use a traditional histogram plot to truly show the results of our simulation. This uses a bulkier sample size: 100,000 total samples. Our goal with this higher sample volume is to attach the t-distribution closely to the stock ticker's historical standard deviation.

The final histogram will subtitle the current stock price, showing bins of the forecasted stock price after one year.

In [27]:
h = go.Figure()
h.add_trace(go.Histogram(
    x = trim, nbinsx = 25,
    marker_color = "rgb(0, 32, 96)", marker_line_color = "black",
    marker_line_width = 1, opacity = 0.95,
    hovertemplate = "<b>Forecasted Price:</b> $%{x:.2f}<br><b>Sample Count:</b> %{y}<extra></extra>"
))
h.update_layout(
    title = dict(text = sym + " — 1-Year Forecasted Price Distribution<br><sup>Current Price: $" + str(round(spot, 2)) + " · 100,000 Simulations</sup>", x = 0.5),
    xaxis_title = "Simulated 1-Year Price", yaxis_title = "Sample Count",
    template = "plotly_white", bargap = 0.15,
    font = dict(family = "Arial", size = 14)
)
h.show()

The above plot concludes our analysis, showing the final forecasted stock price as fitted to a Student's t-distribution. The excess kurtosis shown should model the true market with a higher degree of accuracy, given the high variance of the public markets (Mandelbrot & Hudson, The Misbehavior of Markets, 2004).

###### Citations:

###### Mandelbrot, B., & Hudson, R. L. (2005). The (Mis)Behavior of Markets. The Mathematical Intelligencer, 27(3), 77–79. https://doi.org/10.1007/BF02985848

###### Code cleaned to align with PEP 8 best practices and error troubleshot using Claude generative AI: www.claude.ai. All written content is original and not drafted, edited, or polished using a generative AI system.

### **END OF NOTEBOOK**